# Scraping roguelike articles on the web

This Python notebook describes the data collection process for roguelike universe. If you have not yet installed the requirements, you can do it by running:

`pip install -r requirements.txt`

## Sourcing game titles

We prepared a [list of roguelike games](https://en.wikipedia.org/wiki/List_of_roguelikes) from Wikipedia as a starting point. For indicators of out-of-genre influences, we sourced 10,000+ video game titles from Pastebin as uploaded by the user ________. 

First, we setup read/write functions that are friendly with international unicode characters, because the web may contain all kinds of character code points.

In [1]:
import os
import io
import json
import pandas as pd

def read_json(path):
    data = ''
    with io.open(path, 'r', encoding='utf-8') as f:
        data = json.loads(f.read())
        print(__message('Loaded {}'.format(path)))
    return data
    
def save_json(path, data):
    with io.open(path, 'w', encoding='utf-8') as f:
        try:
            output = json.dumps(data, indent=2, ensure_ascii=False)
            f.write(output)
        except UnicodeEncodeError:
            f.write(output.encode('utf-8'))
    print(__message('Written to {}'.format(path)))
    
def __success(text):
    return '  (SUCC) {}'.format(text).encode('utf-8')
    
def __failure(text):
    return '!!FAIL!! {}'.format(text).encode('utf-8')
    
def __warning(text):
    return '??WARN?? {}'.format(text).encode('utf-8')
    
def __message(text):
    return '   |MSG| {}'.format(text).encode('utf-8')

Here is a sample of the list of roguelike games from Wikipedia:

In [2]:
roguelikes = pd.read_csv(os.path.join(os.getcwd(), 'roguelikes.csv'), skip_blank_lines=True)
roguelikes.head()

,Name,RogueTemple,Link,Status,Released,Updated,Developer,Theme,Influences
0,100 Heroes: Shopkeeper of Doom,http://roguebasin.roguelikedevelopment.org/ind...,http://www.bay12forums.com/smf/index.php?topic...,alpha,2012/00/00,2012/10/25,Paul Wright,Economics/Trading,Recettear
1,100 Rogues,http://roguebasin.roguelikedevelopment.org/ind...,http://www.100rogues.com/,stable,2010/05/06,2010/05/06,Dinofarm Games,Fantasy,Rogue
2,1Quest,http://roguebasin.roguelikedevelopment.org/ind...,http://www.ratzngodz.fr/,stable,2014/02/20,2015/02/07,Ratz 'N' Godz,Fantasy,"DCSS, Dominion4"
3,3059,http://roguebasin.roguelikedevelopment.org/ind...,https://sites.google.com/site/free3069/3059---...,stable,2005/00/00,2005/06/11,Phr00t,"Science Fiction, Alien Planets, Futuristic",nethack
4,3069,http://roguebasin.roguelikedevelopment.org/ind...,http://sites.google.com/site/free3069/,stable,2009/07/06,2009/10/06,Phr00t,"Science Fiction, Alien Planets, Futuristic",3059


In addition, roguelike-like games:

In [3]:
roguelikelikes = pd.read_csv(os.path.join(os.getcwd(), 'roguelike-likes.csv'), skip_blank_lines=True)
roguelikelikes.head()

,Name,Released,Updated,Developer,Theme,Influences
0,ToeJam & Earl,1991,NaN,Johnson Voorsanger Productions,Fantasy,NaN
1,Diablo,1996,NaN,Blizzard North,Fantasy,NaN
2,Diablo II,2000,NaN,Blizzard Entertainment,Fantasy,NaN
3,Lost Labyrinth,2001,2011.0,Lost Labyrinth,Fantasy,NaN
4,Strange Adventures In Infinite Space,2002,2004.0,"Rich Carlson, Iikka Keränen",Space science fiction,NaN


And a sample of the list of video games:

In [4]:
video_games = pd.read_json(os.path.join(os.getcwd(), 'games.json'))
video_games.head(10)

,title,year
0,$hop-n-$pree,2009
1,'43 - One Year After,1986
2,'89 Denno Kyusei Uranai,1988
3,'Nam 1965-1975,1991
4,'Splosion Man,2009
5,'Til Death Do Us Part,2013
6,(Almost) Total Mayhem,2011
7,(Not) Just another Space Shooter,2004
8,(T)Raumschiff Surprise - Periode 1,2004
9,*NSYNC Hotline Phone and Fantasy CD-Rom Game,2001


## Building a corpus 

Before we can do any text analysis, we need to build a corpus in which to operate on.

### 1. RogueTemple

RogueTemple Wiki collects a detailed description of roguelike games.

In [5]:
import bs4
import requests

def scrape_mediawiki_url(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36',
    }
    try:
        response = requests.get(url, headers=headers, timeout=(9.1, 12.1))
    except Exception as e:
        print(__failure('Failed to scrape {}'.format(url)))
        print(e)
        return ''
    
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    
    content = [node.text.strip() for node in soup.select('#mw-content-text') if node.text]
    return ''.join(content)

In [6]:
# Sample scrape
content = scrape_mediawiki_url('http://roguebasin.roguelikedevelopment.org/index.php?title=100_Rogues')
print(content)

100 Rogues



Stable game



Developer

Dinofarm Games



Theme

Fantasy



Influences

Rogue



Released

 (?)



Updated

May 6, 2010 (?)



Licensing

Commercial



P. Language





Platforms

iPhone



Interface

Graphical Tiles



Game Length

Medium



Official site of 100 Rogues




100 Rogues is an original Roguelike for the iPhone, iPod Touch and iPad devices.  It features two playable classes with unique skill trees (similar to Diablo), several tilesets, original SNES-style music, and fully animated pixel art.  100 Rogues was developed from scratch for the iPhone OS devices, and has a click to move control scheme rather than a virtual d-pad.

Contents

1 Gameplay
2 Combat
3 Skills
4 Reception


 Gameplay
A game of 100 Rogues is a linear progression through 12 dungeon levels spread across 3 worlds: The Bandit Hole, The Dungeon, and Hell. The first three levels of each world consist of randomly-generated maps initially populated by Mobs of different Monsters. After these Mobs a

### 2. Wikipedia

Searching on Wikipedia can be done with two dozens line of code.

In [7]:
import wikipedia

def scrape_wiki_id(pageid):
    page = wikipedia.page(pageid=pageid)    
    print_wiki_page(page)
    
def scrape_wiki(title):
    try:
        searchstring = title
        page = wikipedia.page(searchstring, auto_suggest=False)
#         print_wiki_page(page)
        return page
    except wikipedia.DisambiguationError:
        try:
            searchstring = '{} (video game)'.format(title).replace(' ', '_')
            page = wikipedia.page(searchstring, auto_suggest=False)
#             print_wiki_page(page)
            return page
        except wikipedia.DisambiguationError:
            try:
                searchstring = '{} (Unix video game)'.format(title).replace(' ', '_')
                page = wikipedia.page(searchstring, auto_suggest=False)
    #             print_wiki_page(page)
                return page
            except:
                print(__warning(u'Wikipedia cannot find "{}"'.format(searchstring)))
        except:
            print(__warning(u'Wikipedia cannot find "{}"'.format(searchstring)))
    except wikipedia.PageError:
        try:
            page = wikipedia.page(title, auto_suggest=False)
#             print_wiki_page(page)
            return page
        except:
            print(__warning(u'Search term "{}" returned nothing'.format(searchstring)))
    
def print_wiki_page(page):
    print(page.title)
    print(page.content)
    print(page.references)  

In [8]:
# Test Wikipedia crawl
print(scrape_wiki('Rogue Legacy'))

<WikipediaPage 'Rogue Legacy'>


### 3. DuckDuckGo

We also source a list of potential interesting webpages via an internet search engine, DuckDuckGo.

In [9]:
import time
import requests
import urllib.parse

def scrape_duckduckgo(keywords, developer=""):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36',
    }
    searchstring = u'"{}" AND {} AND game AND (mortem OR history OR developer OR review)'.format(keywords, developer)
    q = u'http://duckduckgo.com/html/?q={}'.format(urllib.parse.quote(searchstring.encode('utf-8')))
    print(q)
                                                   
    response = requests.get(q, headers=headers, timeout=(9.1, 12.1))
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    
    links = []
    links = [node.get('href') for node in soup.select('a.result__a')]
    return links

In [24]:
# scrape_duckduckgo('Ancient Domains of Mystery', 'Thomas Biskup')

## Scrape the corpus

With the functions above we can collect a corpus.

In [10]:
def scrape(url, title):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36',
    }
    try:
        print(__message('Scraping {} ...'.format(url)))
        response = requests.get(url, headers=headers, timeout=(10.1, 15.1))
    except Exception as e:
        print(__failure('Failed to load {}'.format(url)))
        print(e)
        return None
    
    html = response.text
    if html and any(word in html.lower() for word in ['tutorial']):
        return None
    if html and any(word in html.lower() for word in [title, 'rogue', 'procedural', 'generation', 'mortem', 'review', 'history', 'develop', 'idea', 'inspir']):
        print(__message('Found article'))
        soup = bs4.BeautifulSoup(response.text, 'lxml')
        selections = soup.select('body > p') + soup.select('div > p') + soup.select('table td')
        content = [node.text.strip() for node in selections]
        return ''.join(content)
    return None

## Web scraping -- all games

In [11]:
# For Roguelike games, we build a corpus with RogueTemple and DuckDuckGo
corpus = []
    
for index, roguelike in roguelikes[:531].iterrows():
    print(roguelike)
    title = roguelike['Name']
    if not isinstance(title, str):
        continue
    text = []
    
    rogue_temple = scrape_mediawiki_url(roguelike['RogueTemple'])
    text.append(rogue_temple)

    developers = str(roguelike['Developer']).replace(',', ' OR ')
    links = scrape_duckduckgo(title, developers)
    
    for link in links[:25]:
        if 'roguebasin.roguelikedevelopment.org' in link \
            or 'roguebasin.com' in link \
            or 'wikipedia' in link:
            continue
        content = scrape(link, title)
        if content:
            text.append(content)
    
    corpus.append({"title": title, "text": text})
    
    # Be nice
    if not links:
        print('Sleeping')
        time.sleep(10)
  
save_json('corpus-1.json', corpus)

Name                              100 Heroes: Shopkeeper of Doom
RogueTemple    http://roguebasin.roguelikedevelopment.org/ind...
Link           http://www.bay12forums.com/smf/index.php?topic...
Status                                                     alpha
Released                                              2012/00/00
Updated                                               2012/10/25
Developer                                            Paul Wright
Theme                                          Economics/Trading
Influences                                             Recettear
Name: 0, dtype: object
http://duckduckgo.com/html/?q=%22100%20Heroes%3A%20Shopkeeper%20of%20Doom%22%20AND%20Paul%20Wright%20AND%20game%20AND%20%28mortem%20OR%20history%20OR%20developer%20OR%20review%29
Name                                                  100 Rogues
RogueTemple    http://roguebasin.roguelikedevelopment.org/ind...
Link                                   http://www.100rogues.com/
Status            

b'   |MSG| Scraping https://www.freegamesutopia.com/game/3059/32/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://mp3itune.ru/songs/%D0%9F%D1%80%D0%B8%D0%BA%D0%BE%D0%BB%D1%8B-%D1%87%D0%B0%D1%81%D1%82%D1%8C-38-%D0%A1%D0%BC%D0%B5%D1%88%D0%BD%D1%8B%D0%B5-%D1%82%D0%B0%D0%BD%D1%86%D1%8B/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.blockchain.com/ru/btc/block/0000000000000000000dfd111ac0dbab1f8a04e91d584a3c45068f9f0e5bc86b ...'
b'   |MSG| Scraping https://lemmasoft.renai.us/forums/viewtopic.php?t=26192 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.blockchain.com/btc/block/00000000000000000003972fb3059fb64850692757c35b39ee13eb0b1b982c75 ...'
b'   |MSG| Scraping https://wordbrainsolver.com/tag/word-bound-level-3059-solution/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://armorgames.com/community/thread/4812204/tank-games?page=1 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://tig.wikia.com/wiki/3059 ...'
b'   |MSG| Found artic

b'   |MSG| Found article'
b'   |MSG| Scraping https://vk.com/phr00t ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://twitter.com/phr00t_ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.indiedb.com/games/3079 ...'
b'   |MSG| Found article'
Name                                                        3089
RogueTemple    http://roguebasin.roguelikedevelopment.org/ind...
Link                              http://3089game.wordpress.com/
Status                                                    stable
Released                                              2013/02/02
Updated                                               2014/02/27
Developer                                                 Phr00t
Theme                 Science Fiction, Alien Planets, Futuristic
Influences                         3059, 3069, 3079, Borderlands
Name: 6, dtype: object
http://duckduckgo.com/html/?q=%223089%22%20AND%20Phr00t%20AND%20game%20AND%20%28mortem%20OR%20history%20OR%20developer%20OR%20re

b'   |MSG| Found article'
b'   |MSG| Scraping https://www.giantbomb.com/adom-ii/3030-37679/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://twitter.com/adom_dev/status/1032750213147095040 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.mixcloud.com/roguelikeradio/episode-33-interview-with-thomas-biskup-creator-of-adom/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://podcastfinder.co/podcasts/roguelike-radio/episode-33-interview-with-thomas-biskup-creator-of-adom ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://wn.com/Adom ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://forums.roguetemple.com/index.php?topic=2386.0 ...'
b'!!FAIL!! Failed to load https://forums.roguetemple.com/index.php?topic=2386.0'
HTTPSConnectionPool(host='forums.roguetemple.com', port=443): Max retries exceeded with url: /index.php?topic=2386.0 (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate v

b'   |MSG| Found article'
b'   |MSG| Scraping http://www.omnilexica.com/?q=amoria ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://myst.forumotion.com/t438-list-of-amoria-pqers ...'
b'   |MSG| Scraping https://www.facebook.com/Amoria-463320430538534/ ...'
b'   |MSG| Scraping https://www.amoriabazaar.com/ ...'
b'   |MSG| Found article'
Name                              A False Saint, An Honest Rogue
RogueTemple    http://roguebasin.roguelikedevelopment.org/ind...
Link                                     http://humbit.com/rogue
Status                                                     alpha
Released                                              2013/03/16
Updated                                               2013/04/21
Developer                                          Jeremiah Reid
Theme                                        Wilderness Survival
Influences            real life, this outsideonline.com article.
Name: 15, dtype: object
http://duckduckgo.com/html/?q=%22A%20False%20

...

http://duckduckgo.com/html/?q=%22The%20Royal%20Wedding%22%20AND%20Ondras%20AND%20game%20AND%20%28mortem%20OR%20history%20OR%20developer%20OR%20review%29
b'   |MSG| Scraping https://www.townandcountrymag.com/leisure/drinks/a19853522/royal-wedding-2018-drinking-game/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://www.girlgames.com/the-royal-wedding.html ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.nowtolove.com.au/royals/british-royal-family/royal-wedding-drinking-game-48398 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://ondras.github.io/trw/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.youtube.com/watch?v=1NG5bOj4Hzs ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://www.latimes.com/entertainment/tv/la-et-st-royal-wedding-nightmare-20180517-story.html ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.bustle.com/p/8-royal-wedding-games-you-can-play-while-prince-harry-meghan-markle-tie-the-knot-8080156 ...'
b' 

http://duckduckgo.com/html/?q=%22The%20Temple%20of%20Torment%22%20AND%20Aukustus%20AND%20game%20AND%20%28mortem%20OR%20history%20OR%20developer%20OR%20review%29
Sleeping
Name                                             The Test of Inn
RogueTemple    http://roguebasin.roguelikedevelopment.org/ind...
Link                              http://users.auth.gr/ekchatzi/
Status                                                      beta
Released                                              2009/00/00
Updated                                               2010/05/08
Developer                                   Lefteris Hatzipetros
Theme                                                        NaN
Influences                                         Elder Scrolls
Name: 518, dtype: object
http://duckduckgo.com/html/?q=%22The%20Test%20of%20Inn%22%20AND%20Lefteris%20Hatzipetros%20AND%20game%20AND%20%28mortem%20OR%20history%20OR%20developer%20OR%20review%29
Sleeping
Name               The Thirty One Lieutena

b'   |MSG| Found article'
b'   |MSG| Scraping https://steamcommunity.com/app/259680/discussions/1/630799997254848090/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.rpgwatch.com/forums/showthread.php?t=18416 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://forums.tigsource.com/index.php?topic=12643.0 ...'
b'   |MSG| Scraping http://www.indiegamenews.com/2016/07/rising-runner-puzzle-your-way-through.html ...'
b'   |MSG| Found article'
Name                                                    The cave
RogueTemple    http://roguebasin.roguelikedevelopment.org/ind...
Link           http://roguecentral.org/doryen/libtcod/project...
Status                                                     alpha
Released                                              2010/09/19
Updated                                               2010/09/19
Developer                                                   jice
Theme                                                        NaN
Influences      

b'   |MSG| Found article'
b'   |MSG| Scraping https://tilerogue.in.uptodown.com/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://crpgaddict.blogspot.com/2010/02/rogue-story-and-gameplay.html ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://www.soft57.com/tilerogue/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://yepdownload.com/tilerogue ...'
b'!!FAIL!! Failed to load https://yepdownload.com/tilerogue'
HTTPSConnectionPool(host='yepdownload.com', port=443): Max retries exceeded with url: /tilerogue (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')],)",),))
b'   |MSG| Scraping https://www.gamespedition.com/pc-games/role-playing-games/tilerogue/tilerogue-secrets-and-easter-eggs ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.dosgamesarchive.com/download/rogue ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://rec.games.roguelike.development.narkive.com/ ..

b'   |MSG| Found article'
b'   |MSG| Scraping https://no-clip.com/video/spspE5YUoos/tomenet-tutorial-1-setting-up-your-own-server.html ...'
b'   |MSG| Scraping https://www.thefullwiki.org/ToME_(computer_game) ...'
b'   |MSG| Found article'
b'   |MSG| Written to corpus-1.json'


In [180]:
# For Roguelike-like games, we build a corpus with Wikipedia and DuckDuckGo
# corpus = read_json('corpus-roguelike-like.json')
# if not corpus:
corpus = []
    
for index, roguelike in roguelikelikes.iterrows():
    print(roguelike)
    title = roguelike['Name']
    text = []
    
    page = scrape_wiki(title)
    if page:
        text.append(page.content)

    developers = str(roguelike['Developer']).replace(',', ' OR ')
    links = scrape_duckduckgo(title, developers)
    
    for link in links[:20]:
        if 'roguebasin.roguelikedevelopment.org' in link \
            or 'roguebasin.com' in link \
            or 'wikipedia' in link:
            continue
        content = scrape(link, title)
        if content:
            text.append(content)
    
    corpus.append({"title": title, "text": text})
  
save_json('corpus-roguelike-like.json', corpus)

Name                           ToeJam & Earl
Released                                1991
Updated                                  NaN
Developer     Johnson Voorsanger Productions
Theme                                Fantasy
Influences                               NaN
Name: 0, dtype: object
http://duckduckgo.com/html/?q=%22ToeJam%20%26%20Earl%22%20AND%20Johnson%20Voorsanger%20Productions%20AND%20game%20AND%20%28mortem%20OR%20history%20OR%20developer%20OR%20review%29
b'   |MSG| Scraping http://gaming.wikia.com/wiki/ToeJam_%26_Earl_Productions ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://lastdaydeaf.com/toejam-earldev-johnson-voorsanger-productions-sega-genesis-1991/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://toejamandearl.wikia.com/wiki/ToeJam_%26_Earl_in_Panic_on_Funkotron ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://segaretro.org/ToeJam_%26_Earl ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.facebook.com/pages/ToeJam-Earl-Prod

/usr/local/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


http://duckduckgo.com/html/?q=%22Diablo%22%20AND%20Blizzard%20North%20AND%20game%20AND%20%28mortem%20OR%20history%20OR%20developer%20OR%20review%29
b'   |MSG| Scraping https://www.youtube.com/watch?v=VscdPA6sUkc ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://us.battle.net/forums/en/d3/topic/7415795753 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://idclips.com/rev/blizzard+diablo/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.pcgamer.com/diablo-designer-david-breviks-full-gdc-post-mortem-is-now-online/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.diablowiki.net/Blizzard_North ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.diabloii.net/blog/comments/diablo-3-post-mortem-jay-wilson-pt1 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://ltclip.com/rev/diablo+blizzard/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://gamasutra.com/view/feature/131533/postmortem_blizzards_diablo_ii.php ...'
b'   |MSG

b'   |MSG| Found article'
b'   |MSG| Scraping http://www.g4g.it/2009/10/01/strange-adventures-in-infinite-space/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.gamezebo.com/2010/11/18/strange-adventures-infinite-space-review/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.gamespot.com/reviews/strange-adventures-in-infinite-space-review/1900-2856375/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.myabandonware.com/game/strange-adventures-in-infinite-space-3qy ...'
b'   |MSG| Scraping http://igrotop.com/games/strange_adventures_in_infinite_space/info ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://www.game-ost.ru/games/6212/strange_adventures_in_infinite_space/meta/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://wiki2.org/en/Strange_Adventures_in_Infinite_Space ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://vikimy.com/l-en/Strange_Adventures_In_Infinite_Space ...'
b'   |MSG| Found article'
b'   |MSG| Scrapin

b'   |MSG| Found article'
b'   |MSG| Scraping http://bestgamer.net/load/3125-the-binding-of-isaac-edmund-mcmillen-eng-p.html ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://tcrf.net/The_Binding_of_Isaac ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://steamcommunity.com/workshop/filedetails/?id=895960454 ...'
b'   |MSG| Found article'
Name          FTL: Faster Than Light
Released                        2012
Updated                          NaN
Developer               Subset Games
Theme          Space science fiction
Influences                       NaN
Name: 8, dtype: object
http://duckduckgo.com/html/?q=%22FTL%3A%20Faster%20Than%20Light%22%20AND%20Subset%20Games%20AND%20game%20AND%20%28mortem%20OR%20history%20OR%20developer%20OR%20review%29
b'   |MSG| Scraping http://gaming.wikia.com/wiki/FTL:_Faster_Than_Light ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://speed-new.com/ftl-faster-than-light-full-pc-game ...'
b'   |MSG| Found article'
b'   |MSG| Scrap

...

b'   |MSG| Found article'
Name          The Binding of Isaac: Rebirth
Released                               2014
Updated                                 NaN
Developer                           Nicalis
Theme                               Fantasy
Influences                              NaN
Name: 22, dtype: object
http://duckduckgo.com/html/?q=%22The%20Binding%20of%20Isaac%3A%20Rebirth%22%20AND%20Nicalis%20AND%20game%20AND%20%28mortem%20OR%20history%20OR%20developer%20OR%20review%29
b'   |MSG| Scraping https://gamesmojo.com/games/action/the-binding-of-isaac-rebirth ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://maddownload.com/games/rpg/the-binding-of-isaac-rebirth/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://store.steampowered.com/app/250900/The_Binding_of_Isaac_Rebirth/ ...'
b'   |MSG| Scraping http://fragrun.com/games/action/the-binding-of-isaac-rebirth ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://playstationgamer.org/games/action/the-binding-of

b'   |MSG| Scraping https://wccftech.com/review/hand-of-fate-2/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://oceanofgames.com/hand-of-fate-free-download/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.gamespot.com/reviews/hand-of-fate-2-review/1900-6416814/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.gog.com/game/hand_of_fate_2_outlands_and_outsiders ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://www.gamerevolution.com/review/67003-hand-of-fate-review ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://gamestorrent.co/hand-of-fate-2-outlands-and-outsiders-plaza.html ...'
b'   |MSG| Scraping https://www.pcpowerplay.com.au/review/hand-of-fate-2,477783 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://speed-new.com/hand-of-fate-full-pc-game ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://unity3d.com/showcase/case-stories/hand-of-fate ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.facebook.

b'??WARN?? Search term "Infinite Space III: Sea of Stars" returned nothing'
http://duckduckgo.com/html/?q=%22Infinite%20Space%20III%3A%20Sea%20of%20Stars%22%20AND%20Rich%20Carlson%20OR%20%20Iikka%20Ker%C3%A4nen%20AND%20game%20AND%20%28mortem%20OR%20history%20OR%20developer%20OR%20review%29
b'   |MSG| Scraping https://www.imdb.com/title/tt1323932/fullcredits ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.facebook.com/WeirdWorldsReturnToInfiniteSpace ...'
b'   |MSG| Scraping https://www.facebook.com/infinitespace3 ...'
b'   |MSG| Scraping http://doom.wikia.com/wiki/List_of_notable_WADs ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://doomwiki.org/wiki/Hacx ...'
b'   |MSG| Scraping https://doomwiki.org/wiki/Requiem ...'
b'   |MSG| Scraping https://store.steampowered.com/app/698100/Protagon_VR/ ...'
b'   |MSG| Scraping http://doom.wikia.com/wiki/Requiem ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://onemandoom.blogspot.com/p/index-of-reviews.html ...'


b'   |MSG| Found article'
b'   |MSG| Scraping https://steamcommunity.com/app/265000/discussions/0/365163537817753926/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.steamgifts.com/discussion/VYsDF/something-to-read-interesting-history-of-indie-dev-betadwarf-makers-of-forced-and-forced-showdown ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.destructoid.com/review-forced-showdown-368410.phtml ...'
b'   |MSG| Scraping https://ru-ru.facebook.com/BetaDwarf/posts ...'
b'   |MSG| Scraping http://cogconnected.com/review/forced-showdown-review/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://linuxgamenews.com/post/63208838351/the-incredible-story-of-betadwarf-and-forced ...'
b'   |MSG| Scraping https://gamingbolt.com/forced-review ...'
b'   |MSG| Scraping http://www.grabthegames.com/review-forced-showdown.html ...'
b'   |MSG| Found article'
Name          Pixel Cave
Released            2016
Updated              NaN
Developer       Megabyte
Theme         

b'   |MSG| Scraping https://saveorquit.com/2017/10/07/review-heat-signature/ ...'
b'   |MSG| Scraping http://heatsig.com/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.rockpapershotgun.com/2017/09/26/heat-signature-review/ ...'
b'   |MSG| Scraping https://gamesmojo.com/games/action/heat-signature ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://heatsignature.wikia.com/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.pcgamesn.com/heat-signature/heat-signature-release-date-steam-trading-cards ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.polygon.com/2017/9/22/16350330/heat-signature-review ...'
b'   |MSG| Scraping https://waypoint.vice.com/en_us/article/evpvmk/watch-us-steal-cool-spaceships-in-heat-signature ...'
b'   |MSG| Found article'
Name                   FARA
Released               2018
Updated                 NaN
Developer     Brian Roberts
Theme               Fantasy
Influences              NaN
Name: 37, dtype: object
b'?

## Web scraping - subset of games

Because we don't have all the time in the world.

In [222]:
# For Roguelike games, we build a corpus with RogueTemple and DuckDuckGo
corpus = []
games_to_scrape = ["The Tombs"]
    
for index, roguelike in roguelikes.iterrows():
    title = roguelike['Name']
    if not isinstance(title, str) or not title in games_to_scrape:
        continue
    print(roguelike)
    text = []
    
    rogue_temple = scrape_mediawiki_url(roguelike['RogueTemple'])
    text.append(rogue_temple)

    developers = '"' + str(roguelike['Developer']).replace(',', ' OR ') + '"'
    links = scrape_duckduckgo(title, developers)
    
    for link in links[:25]:
        if 'roguebasin.roguelikedevelopment.org' in link \
            or 'roguebasin.com' in link \
            or 'wikipedia' in link:
            continue
        content = scrape(link, title)
        if content:
            text.append(content)
    
    corpus.append({"title": title, "text": text})
  
save_json('corpus-addition.tmp.json', corpus)

Name                                                   The Tombs
RogueTemple    http://roguebasin.roguelikedevelopment.org/ind...
Link                                    http://www.thetombs.com/
Released                                              2005/04/16
Updated                                               2005/04/16
Developer                                         Martin Woodard
Theme                                                    Fantasy
Influences                                                   NaN
Name: 229, dtype: object
http://duckduckgo.com/html/?q=%22The%20Tombs%22%20AND%20%22Martin%20Woodard%22%20AND%20game%20AND%20%28mortem%20OR%20history%20OR%20developer%20OR%20review%29
b'   |MSG| Scraping http://time.com/4791258/game-of-thrones-george-r-r-martin-interview/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.theverge.com/2018/4/25/17280908/george-r-r-martin-grrm-game-of-thrones-song-of-ice-and-fire ...'
b'   |MSG| Found article'
b'   |MSG| Scraping h

KeyboardInterrupt: 